In [27]:
from scipy.io import loadmat
import numpy as np
import nibabel as nib

In [28]:
mpc_mat = loadmat('./cingulate_mat/MPC.mat')
mpc = mpc_mat['MPC2']
cingulatefs5_mask = loadmat('./cingulate_mat/cingulate_maskfsa5.mat')
economo_mask = loadmat('./cingulate_mat/voneconomo.mat')
schaefer1000 = nib.load("Schaefer2018_1000Parcels_7Networks_order.dscalar.nii")
#print(mpc)
meanMPC = np.squeeze(np.mean(mpc,1))

In [35]:
# make gradient
Gp = GradientMaps(kernel='normalized_angle', approach='dm')
Gp = Gp.fit(meanMPC)

In [43]:
tmp = np.zeros(20484)
for i in range(1,500,1):
    result = np.where(schaefer1000 == i+1, [tmp]) 
    result = Gp.gradients_[1](i,1)

ValueError: either both or neither of x and y should be given

In [16]:
from brainspace.datasets import load_conte69, load_parcellation
from brainspace.gradient import GradientMaps
from brainspace.plotting import plot_hemispheres
from brainspace.utils.parcellation import map_to_labels
# and load the conte69 hemisphere surfaces
surf_lh, surf_rh = load_conte69()
# First load mean connectivity matrix and parcellation

labeling = load_parcellation('schaefer', scale=400, join=True)
mask = labeling != 0

seeds = [None] * 2
#seeds[0] = map_to_labels(fc[0], labeling, mask=mask, fill=np.nan)
seeds[0] = map_to_labels(mpc[0], labeling, mask=mask, fill=np.nan)

# visualise the features from a seed region (seed 0)
plot_hemispheres(surf_lh, surf_rh, array_name=seeds, label_text=['MPC'],
                 size=(1200, 400), color_bar=True, cmap='viridis', zoom=1.45)

KeyError: 0